<h1>Capstone Project - The Battle of Singapore Neighborhoods</h1>

In [5]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

<h1> Download and explore the data set of Singapore residents</h1>

In [6]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d957dcffc6fa47cbbc3bb5b958fec52b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='wwZ_zzoRWaa4CqlEj7dLEWns-xHUF27I0j01-tM2Z6Lj',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d957dcffc6fa47cbbc3bb5b958fec52b.get_object(Bucket='courseracapstoneproject-donotdelete-pr-ublxjzfmhfybtw',Key='SG_residents4.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_sg_residents = pd.read_csv(body)
df_sg_residents

,Borough,Neighborhood,Latitude,Longitude,Average no of residents
0,Ang Mo Kio,Ang Mo Kio Town Centre,1.371285,103.846994,1899505
1,Bedok,Bedok North,1.331920,103.924744,1942315
2,Clementi,Clementi Central,1.315100,103.765231,1906745
3,Downtown Core,Anson,1.272658,103.843801,0
4,Sembawang,Sembawang Central,1.449093,103.820056,1922410
5,Bukit Panjang,Bangkit,1.379323,103.772566,1910695
6,Bukit Batok,Bukit Batok Central,1.349091,103.749619,1911280
7,Sengkang,Anchorvale,1.397140,103.886659,1923095
8,Outram,Chinatown,1.283737,103.843798,1906700
9,Museum,Fort Canning,1.292383,103.844242,1900670


In [7]:
df_sg_residents.dtypes

Borough                     object
Neighborhood                object
Latitude                   float64
Longitude                  float64
Average no of residents      int64
dtype: object

<h1>Use geopy library to get the latitude and longitude values of Singapore.</h1>

In [8]:
from geopy.geocoders import Nominatim
address = 'Singapore'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


<h1>Create a map of Singapore with 5 neighborhoods superimposed on top.</h1>

In [9]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [10]:
# create map of Singapore using latitude and longitude values
map_singapore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_sg_residents['Latitude'], df_sg_residents['Longitude'],df_sg_residents['Neighborhood'],df_sg_residents['Average no of residents']):
    label='{}, {}'.format(neighborhood, borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_singapore)
map_singapore

<h1>Define Foursquare Credentials and Version</h1>

In [11]:
CLIENT_ID = 'GG2Z0YB43XC2TNEMJDXQ1ZNPIZWELQG1SEZSSTB3VJHMKJWJ' # your Foursquare ID
CLIENT_SECRET = 'OD42ZZ4PLSYKEIMDI34B2BH3QYNP420RESVRZLHQ0ZXG4JSU' # your Foursquare Secret
VERSION = '20200425' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GG2Z0YB43XC2TNEMJDXQ1ZNPIZWELQG1SEZSSTB3VJHMKJWJ
CLIENT_SECRET:OD42ZZ4PLSYKEIMDI34B2BH3QYNP420RESVRZLHQ0ZXG4JSU


In [12]:
df_sg_residents.loc[0, 'Neighborhood']

'Ang Mo Kio Town Centre'

<h1>Get Ang Mo Kio’s latitude and longitude values.</h1>

In [13]:
neighborhood_latitude = df_sg_residents.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_sg_residents.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = df_sg_residents.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,neighborhood_latitude,neighborhood_longitude))

Latitude and longitude values of Ang Mo Kio Town Centre are 1.3712845, 103.8469936.


<h1>Get the top 100 venues that are in Ang Mo Kio Town Centre within a radius of 500 meters.</h1>

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

In [15]:
radius = 500 # define radius

In [16]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GG2Z0YB43XC2TNEMJDXQ1ZNPIZWELQG1SEZSSTB3VJHMKJWJ&client_secret=OD42ZZ4PLSYKEIMDI34B2BH3QYNP420RESVRZLHQ0ZXG4JSU&v=20200425&ll=1.3712845,103.8469936&radius=500&limit=100'

<h1>Send the GET request and examine the results</h1>

In [17]:
import requests # library to handle requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea5864fda9e14001b10271b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Ang Mo Kio',
  'headerFullLocation': 'Ang Mo Kio, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 1.3757845045000046,
    'lng': 103.85148648919906},
   'sw': {'lat': 1.3667844954999955, 'lng': 103.84250071080095}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50712b67e4b0548504446dea',
       'name': 'Face Ban Mian 非板面 (Ang Mo Kio)',
       'location': {'address': '722 Ang Mo Kio Ave 8',
        'crossStreet': 'Ang Mo Kio Central',
        'lat': 1.3720308862082362,
        'lng': 103.84750442817865

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h1>Clean the json and structure it into a pandas dataframe.</h1>

In [19]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Face Ban Mian 非板面 (Ang Mo Kio),Noodle House,1.372031,103.847504
1,NTUC FairPrice,Supermarket,1.371507,103.847082
2,Old Chang Kee,Snack Place,1.369094,103.848389
3,FairPrice Xtra,Supermarket,1.369279,103.848886
4,True Fitness,Gym,1.372891,103.847661


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


<h1>Explore Neighborhoods in Singapore</h1>

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h1>Run the above function on each neighborhood and create a new
dataframe called singapore_venues.</h1>

In [22]:
singapore_venues = getNearbyVenues(names=df_sg_residents['Neighborhood'],
                                   latitudes=df_sg_residents['Latitude'],
                                   longitudes=df_sg_residents['Longitude']
                                  )

Ang Mo Kio Town Centre
Bedok North
Clementi Central
Anson
Sembawang Central
Bangkit
Bukit Batok Central
Anchorvale
Chinatown
Fort Canning


In [23]:
print(singapore_venues.shape)
singapore_venues.head()

(471, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ang Mo Kio Town Centre,1.371285,103.846994,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House
1,Ang Mo Kio Town Centre,1.371285,103.846994,NTUC FairPrice,1.371507,103.847082,Supermarket
2,Ang Mo Kio Town Centre,1.371285,103.846994,Old Chang Kee,1.369094,103.848389,Snack Place
3,Ang Mo Kio Town Centre,1.371285,103.846994,FairPrice Xtra,1.369279,103.848886,Supermarket
4,Ang Mo Kio Town Centre,1.371285,103.846994,True Fitness,1.372891,103.847661,Gym


In [24]:
singapore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Anchorvale,12,12,12,12,12,12
Ang Mo Kio Town Centre,48,48,48,48,48,48
Anson,81,81,81,81,81,81
Bangkit,7,7,7,7,7,7
Bedok North,14,14,14,14,14,14
Bukit Batok Central,26,26,26,26,26,26
Chinatown,100,100,100,100,100,100
Clementi Central,71,71,71,71,71,71
Fort Canning,96,96,96,96,96,96


In [25]:
print('There are {} uniques categories.'.format(len(singapore_venues['Venue Category'].unique())))

There are 141 uniques categories.


<h1>Analyze Each Neighborhood</h1>

In [26]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Neighborhood'] = singapore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()

,Neighborhood,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bike Trail,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Cantonese Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Flea Market,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nightclub,Noodle House,Other Great Outdoors,Park,Pedestrian Plaza,Persian Restaurant,Pet Store,Pizza Place,Playground,Pool,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,River,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soba Restaurant,Soup Place,Spa,Spanish Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Park Ride / Attraction,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Ang Mo Kio Town Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ang Mo Kio Town Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ang Mo Kio Town Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ang Mo Kio Town Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ang Mo Kio Town Centre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
singapore_onehot.shape

(471, 142)

In [28]:
singapore_grouped = singapore_onehot.groupby('Neighborhood').mean().reset_index()
singapore_grouped

,Neighborhood,ATM,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bike Trail,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Bus Station,Bus Stop,Café,Cantonese Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Fast Food Restaurant,Flea Market,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lighthouse,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nightclub,Noodle House,Other Great Outdoors,Park,Pedestrian Plaza,Persian Restaurant,Pet Store,Pizza Place,Playground,Pool,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,River,Sake Bar,Salad Place,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soba Restaurant,Soup Place,Spa,Spanish Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theme Park Ride / Attraction,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Anchorvale,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.083333,0.00,0.083333,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00
1,Ang Mo Kio Town Centre,0.000000,0.000000,0.000000,0.000000,0.020833,0.00,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.041667,0.00,0.000000,0.020833,0.000000,0.000000,0.000000,0.00,0.000000,0.020833,0.00,0.000000,0.083333,0.020833,0.00,0.000000,0.062500,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.020833,0.000000,0.041667,0.00,0.125000,0.00,0.020833,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.020833,0.020833,0.000000,0.020833,0.000000,0.020833,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.000000,0.000000,

In [29]:
singapore_grouped.shape

(10, 142)

<h1>Show each neighborhood along with the top 10 most common venues</h1>

In [30]:
num_top_venues = 10

for hood in singapore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = singapore_grouped[singapore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anchorvale----
                  venue  freq
0    Italian Restaurant  0.08
1      Basketball Court  0.08
2                  Park  0.08
3  Fast Food Restaurant  0.08
4            Food Court  0.08
5                Bridge  0.08
6                   Gym  0.08
7  Gym / Fitness Center  0.08
8           Beer Garden  0.08
9          Hockey Arena  0.08


----Ang Mo Kio Town Centre----
                  venue  freq
0            Food Court  0.12
1           Coffee Shop  0.08
2          Dessert Shop  0.06
3    Seafood Restaurant  0.04
4           Supermarket  0.04
5           Snack Place  0.04
6       Bubble Tea Shop  0.04
7   Japanese Restaurant  0.04
8  Fast Food Restaurant  0.04
9      Malay Restaurant  0.02


----Anson----
                 venue  freq
0  Japanese Restaurant  0.12
1          Coffee Shop  0.11
2                 Café  0.07
3                Hotel  0.06
4           Food Court  0.04
5     Ramen Restaurant  0.04
6      Soba Restaurant  0.02
7           Soup Place  0.02
8          

<h1>Put them into a pandas dataframe and sort the venues in descending order</h1>

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
import numpy as np # library to handle data in a vectorized manner

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anchorvale,Food Court,Beer Garden,Hockey Arena,Park,Pool,Italian Restaurant,Fast Food Restaurant,Trail,Gym / Fitness Center,Gym
1,Ang Mo Kio Town Centre,Food Court,Coffee Shop,Dessert Shop,Bubble Tea Shop,Japanese Restaurant,Seafood Restaurant,Supermarket,Fast Food Restaurant,Snack Place,Breakfast Spot
2,Anson,Japanese Restaurant,Coffee Shop,Café,Hotel,Food Court,Ramen Restaurant,Salad Place,Soup Place,Sushi Restaurant,Bar
3,Bangkit,Food Court,Bike Trail,Park,Noodle House,Market,Miscellaneous Shop,Yoga Studio,Electronics Store,Flea Market,Fast Food Restaurant
4,Bedok North,Food Court,Coffee Shop,Playground,Thai Restaurant,Park,Bus Station,Fast Food Restaurant,Steakhouse,Supermarket,Convenience Store
5,Bukit Batok Central,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Food Court,Malay Restaurant,Café,Multiplex,Sandwich Place,Shopping Mall,Mobile Phone Shop
6,Chinatown,Chinese Restaurant,Hostel,Food Court,French Restaurant,Korean Restaurant,Vegetarian / Vegan Restaurant,Spa,Café,Japanese Restaurant,Burger Joint
7,Clementi Central,Coffee Shop,Food Court,Bus Station,Asian Restaurant,Chinese Restaurant,Dessert Shop,Electronics Store,Shopping Mall,Chinese Breakfast Place,Snack Place
8,Fort Canning,Japanese Restaurant,Hotel,Nightclub,Bar,Pub,Spa,Bakery,Pizza Place,Ramen Restaurant,Restaurant
9,Sembawang Central,Convenience Store,Coffee Shop,Shopping Mall,Asian Restaurant,Japanese Restaurant,Fast Food Restaurant,Wings Joint,Chinese Restaurant,Bistro,Supermarket


<h1>Run k-means to cluster the neighborhood into 5 clusters.</h1>

In [33]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

singapore_grouped_clustering = singapore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 4, 3, 2, 2, 4, 2, 4, 0], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#singapore_merged = df_sg_residents
# merge neighborhood_venues_sorted with df_sg_residents to add latitude/longitude for each neighborhood
singapore_merged = df_sg_residents.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

singapore_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Average no of residents,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ang Mo Kio,Ang Mo Kio Town Centre,1.371285,103.846994,1899505,2,Food Court,Coffee Shop,Dessert Shop,Bubble Tea Shop,Japanese Restaurant,Seafood Restaurant,Supermarket,Fast Food Restaurant,Snack Place,Breakfast Spot
1,Bedok,Bedok North,1.331920,103.924744,1942315,2,Food Court,Coffee Shop,Playground,Thai Restaurant,Park,Bus Station,Fast Food Restaurant,Steakhouse,Supermarket,Convenience Store
2,Clementi,Clementi Central,1.315100,103.765231,1906745,2,Coffee Shop,Food Court,Bus Station,Asian Restaurant,Chinese Restaurant,Dessert Shop,Electronics Store,Shopping Mall,Chinese Breakfast Place,Snack Place
3,Downtown Core,Anson,1.272658,103.843801,0,4,Japanese Restaurant,Coffee Shop,Café,Hotel,Food Court,Ramen Restaurant,Salad Place,Soup Place,Sushi Restaurant,Bar
4,Sembawang,Sembawang Central,1.449093,103.820056,1922410,0,Convenience Store,Coffee Shop,Shopping Mall,Asian Restaurant,Japanese Restaurant,Fast Food Restaurant,Wings Joint,Chinese Restaurant,Bistro,Supermarket
5,Bukit Panjang,Bangkit,1.379323,103.772566,1910695,3,Food Court,Bike Trail,Park,Noodle House,Market,Miscellaneous Shop,Yoga Studio,Electronics Store,Flea Market,Fast Food Restaurant
6,Bukit Batok,Bukit Batok Central,1.349091,103.749619,1911280,2,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Food Court,Malay Restaurant,Café,Multiplex,Sandwich Place,Shopping Mall,Mobile Phone Shop
7,Sengkang,Anchorvale,1.397140,103.886659,1923095,1,Food Court,Beer Garden,Hockey Arena,Park,Pool,Italian Restaurant,Fast Food Restaurant,Trail,Gym / Fitness Center,Gym
8,Outram,Chinatown,1.283737,103.843798,1906700,4,Chinese Restaurant,Hostel,Food Court,French Restaurant,Korean Restaurant,Vegetarian / Vegan Restaurant,Spa,Café,Japanese Restaurant,Burger Joint
9,Museum,Fort Canning,1.292383,103.844242,1900670,4,Japanese Restaurant,Hotel,Nightclub,Bar,Pub,Spa,Bakery,Pizza Place,Ramen Restaurant,Restaurant


In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['Neighborhood'], singapore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Examine the 5 clusters to identify segmentation and locations that will achieve the highest potential in revenue and economies of scale for a business </h1>

In [36]:
df_cluster_0=(singapore_merged.loc[singapore_merged['Cluster Labels'] == 0, singapore_merged.columns[[1] + list(range(2, singapore_merged.shape[1]))]])
df_cluster_0

,Neighborhood,Latitude,Longitude,Average no of residents,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Sembawang Central,1.449093,103.820056,1922410,0,Convenience Store,Coffee Shop,Shopping Mall,Asian Restaurant,Japanese Restaurant,Fast Food Restaurant,Wings Joint,Chinese Restaurant,Bistro,Supermarket


In [37]:
df_cluster_1=(singapore_merged.loc[singapore_merged['Cluster Labels'] == 1, singapore_merged.columns[[1] + list(range(2, singapore_merged.shape[1]))]])
df_cluster_1

,Neighborhood,Latitude,Longitude,Average no of residents,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Anchorvale,1.39714,103.886659,1923095,1,Food Court,Beer Garden,Hockey Arena,Park,Pool,Italian Restaurant,Fast Food Restaurant,Trail,Gym / Fitness Center,Gym


In [38]:
df_cluster_2=(singapore_merged.loc[singapore_merged['Cluster Labels'] == 2, singapore_merged.columns[[1] + list(range(2, singapore_merged.shape[1]))]])
df_cluster_2

,Neighborhood,Latitude,Longitude,Average no of residents,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ang Mo Kio Town Centre,1.371285,103.846994,1899505,2,Food Court,Coffee Shop,Dessert Shop,Bubble Tea Shop,Japanese Restaurant,Seafood Restaurant,Supermarket,Fast Food Restaurant,Snack Place,Breakfast Spot
1,Bedok North,1.331920,103.924744,1942315,2,Food Court,Coffee Shop,Playground,Thai Restaurant,Park,Bus Station,Fast Food Restaurant,Steakhouse,Supermarket,Convenience Store
2,Clementi Central,1.315100,103.765231,1906745,2,Coffee Shop,Food Court,Bus Station,Asian Restaurant,Chinese Restaurant,Dessert Shop,Electronics Store,Shopping Mall,Chinese Breakfast Place,Snack Place
6,Bukit Batok Central,1.349091,103.749619,1911280,2,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Food Court,Malay Restaurant,Café,Multiplex,Sandwich Place,Shopping Mall,Mobile Phone Shop


In [39]:
df_cluster_3=(singapore_merged.loc[singapore_merged['Cluster Labels'] == 3, singapore_merged.columns[[1] + list(range(2, singapore_merged.shape[1]))]])
df_cluster_3

,Neighborhood,Latitude,Longitude,Average no of residents,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Bangkit,1.379323,103.772566,1910695,3,Food Court,Bike Trail,Park,Noodle House,Market,Miscellaneous Shop,Yoga Studio,Electronics Store,Flea Market,Fast Food Restaurant


In [40]:
df_cluster_4=(singapore_merged.loc[singapore_merged['Cluster Labels'] == 4, singapore_merged.columns[[1] + list(range(2, singapore_merged.shape[1]))]])
df_cluster_4

,Neighborhood,Latitude,Longitude,Average no of residents,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Anson,1.272658,103.843801,0,4,Japanese Restaurant,Coffee Shop,Café,Hotel,Food Court,Ramen Restaurant,Salad Place,Soup Place,Sushi Restaurant,Bar
8,Chinatown,1.283737,103.843798,1906700,4,Chinese Restaurant,Hostel,Food Court,French Restaurant,Korean Restaurant,Vegetarian / Vegan Restaurant,Spa,Café,Japanese Restaurant,Burger Joint
9,Fort Canning,1.292383,103.844242,1900670,4,Japanese Restaurant,Hotel,Nightclub,Bar,Pub,Spa,Bakery,Pizza Place,Ramen Restaurant,Restaurant


<h2> Based on the results, it is interesting to compare the tradeoffs and advantages between Cluster 2 & 4 in terms of the distance between each similar neighborhoods and the total average no of residents (ie demand volume).</h2>